In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Word2Vec
import timeit 

In [2]:
# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.131:7077") \
        .appName("experiment_2.ipynb")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.cores.max",2)\
        .config("spark.worker.instances",5)\
        .config("spark.executor.cores",1)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()
#\#)\
# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")
print("Started Session")

Started Session


In [3]:
def create_spark_dataframe(filename):
    df = spark_session.read.json('hdfs://192.168.2.131:9000/user/ubuntu/{}'.format(filename))
    return df

In [4]:
start_time_0 = timeit.default_timer() 

df = create_spark_dataframe('RC_2005-12.json')
# df = create_spark_dataframe('RC_2011-08.json')
# df = create_spark_dataframe('RC_2009-05.json')

df.printSchema()

print("execution time {}s".format(timeit.default_timer()-start_time_0))

root
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: long (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: boolean (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)

execution time 17.098277665965725s


In [5]:
comments = df.select("body").withColumnRenamed("body", "comment").rdd

In [6]:
comments.take(2)

[Row(comment='A look at Vietnam and Mexico exposes the myth of market liberalisation.'),
 Row(comment='The site states "What can I use it for? Meeting notes, Reports, technical specs Sign-up sheets, proposals and much more...", just like any other new breeed of sites that want us to store everything we have on the web. And they even guarantee multiple levels of security and encryption etc. But what prevents these web site operators fom accessing and/or stealing Meeting notes, Reports, technical specs Sign-up sheets, proposals and much more, for competitive or personal gains...? I am pretty sure that most of them are honest, but what\'s there to prevent me from setting up a good useful site and stealing all your data? Call me paranoid - I am.')]

In [7]:
comments = comments.map(lambda x: [x[0].split()]).toDF(["comment"])

In [8]:
comments.take(2)

[Row(comment=['A', 'look', 'at', 'Vietnam', 'and', 'Mexico', 'exposes', 'the', 'myth', 'of', 'market', 'liberalisation.']),
 Row(comment=['The', 'site', 'states', '"What', 'can', 'I', 'use', 'it', 'for?', 'Meeting', 'notes,', 'Reports,', 'technical', 'specs', 'Sign-up', 'sheets,', 'proposals', 'and', 'much', 'more...",', 'just', 'like', 'any', 'other', 'new', 'breeed', 'of', 'sites', 'that', 'want', 'us', 'to', 'store', 'everything', 'we', 'have', 'on', 'the', 'web.', 'And', 'they', 'even', 'guarantee', 'multiple', 'levels', 'of', 'security', 'and', 'encryption', 'etc.', 'But', 'what', 'prevents', 'these', 'web', 'site', 'operators', 'fom', 'accessing', 'and/or', 'stealing', 'Meeting', 'notes,', 'Reports,', 'technical', 'specs', 'Sign-up', 'sheets,', 'proposals', 'and', 'much', 'more,', 'for', 'competitive', 'or', 'personal', 'gains...?', 'I', 'am', 'pretty', 'sure', 'that', 'most', 'of', 'them', 'are', 'honest,', 'but', "what's", 'there', 'to', 'prevent', 'me', 'from', 'setting', 'up'

In [9]:
word2Vec = Word2Vec(vectorSize=4, minCount=4, inputCol="comment", outputCol="result")
model = word2Vec.fit(comments)

In [19]:
synonyms = model.findSynonyms('pretty', 5)

In [20]:
for word, cosine_distance in synonyms.collect():
    print(word, cosine_distance)

fact 0.9996135830879211
it, 0.9976227283477783
thanks 0.9950999617576599
movies 0.9936641454696655
new 0.9933768510818481


In [28]:
# release the cores for another application!
spark_context.stop()
print("Ended Session")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:43105)
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/context.py:460: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed or may also be in a zombie state.
  warnings.warn(


Ended Session
